In [223]:
import pandas as pd
import re
import emoji
import spacy
# Spacy's english model
nlp = spacy.load('en_core_web_sm')

from bertopic import BERTopic

from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

In [224]:
# Filepaths
tweet_data_fp = 'twitter_data/custom_data/pruned_medicine_and_research_users.csv'

In [225]:
tweet_data_df = pd.read_csv(tweet_data_fp)
tweet_data_df = tweet_data_df[tweet_data_df['Lang_code'] == 'en']

## Preprocessing

In [226]:
# Gathering documents
docs = tweet_data_df['Tweet'].tolist()
pruned_docs = docs

In [227]:
# Removal of links
def remove_urls(doc):
    return re.sub(r'http\S+', '', doc)

pruned_docs = [remove_urls(doc) for doc in pruned_docs]

# Converting emojis into words
def convert_emojis(doc):
    # delimiters are what is used around the emoji description, in this case spaces are used
    return emoji.demojize(doc, delimiters=(" ", " "))

#pruned_docs = [convert_emojis(doc) for doc in pruned_docs]

# Converting hashtags into words
# Or removing the hashtag and word entirely
def remove_hashtags(doc):
    return doc.replace('#', '')
    #return re.sub(r'#\w+', '', doc)

#pruned_docs = [remove_hashtags(doc) for doc in pruned_docs]

# Converting mentions into words
def remove_numbers(doc):
    return re.sub(r'\d+', '', doc)

pruned_docs = [remove_numbers(doc) for doc in pruned_docs]

# Remove punctuation 
def remove_punctuation(doc):
    return re.sub(r'[^\w\s]', '', doc)

#pruned_docs = [remove_punctuation(doc) for doc in pruned_docs]

# Lemmatize the text
def lemmatize_text(doc):
    doc = nlp(doc)
    return ' '.join([token.lemma_ for token in doc])

pruned_docs = [lemmatize_text(doc) for doc in pruned_docs]

In [228]:
tokenizer = lambda s: re.findall( '\w+', s.lower() )

pruned_docs_split = [ tokenizer(doc) for doc in  pruned_docs ]

# Combine the inner lists into sentences
pruned_docs_tokenized = [' '.join(words) for words in pruned_docs_split]

In [229]:
pruned_docs_tokenized

['speak of kids and infection',
 'in practice by announce a price tag notably during the deep global recession since the second world war nature have fail to enact their own diversity and inclusion pledge nail it dr pai',
 'awesome',
 'good',
 'out with a whimper',
 'steady tuesday twitterverse',
 'one recent study suggest that the families first coronavirus response act ffcra which grant some worker pay sick leave or expand family and medical leave due to covid helped reduce the spread of the virus learn more',
 'anyone look for poster idea to encourage vaccination in your hospital or clinic',
 'tomorrow be december brace yourself year in review article be come',
 'just over at the neighbor and if they re out of what we need the key to their pickup be under the seat so we drive to the supermarket',
 'akk sorry folk this be live a week from tomorrow i erroneously tweet that it be tomorrow apology to all it be still two great speaker and a moderator who have lose track of time',
 'ameri

## Performing BERTopic

In [230]:
# Topic modelling
bertopic_model = BERTopic()
topics, probs = bertopic_model.fit_transform(pruned_docs_tokenized)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [231]:
pruned_topic_df = bertopic_model.get_document_info(pruned_docs)

In [232]:
pruned_topic_df['Topic'].nunique()

292

## Calculating Topic Coherence

In [233]:
topic_info = bertopic_model.get_topic_info()
topics = topic_info['Representation'].tolist()

word2id = Dictionary(pruned_docs_split)

In [234]:
cm = CoherenceModel(topics=topics, 
                    texts=pruned_docs_split,
                    coherence='c_v',  # c_npmi was used in the bertopic serbian research paper
                    dictionary=word2id)

In [235]:
coherence_per_topic = cm.get_coherence_per_topic()
macro_topic_coherence = sum(coherence_per_topic) / len(coherence_per_topic)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [236]:
# This returns 0.02 when set to npmi, the serbian research return -0.042 as their best score
# This returns 0.511 when set to c_v, this is a good coherence score
# When removing non-english words and links this drops to 0.44
# When adding lemmatization this goes up to 0.46
# Removing numbers drop to 0.4567
macro_topic_coherence 

0.45675962412417664

## Calculating Topic Diversity

In [106]:
#Found here: https://github.com/silviatti/topic-model-diversity/blob/master/diversity_metrics.py
def proportion_unique_words(topics, topk=10):
    """
    compute the proportion of unique words

    Parameters
    ----------
    topics: a list of lists of words
    topk: top k words on which the topic diversity will be computed (There are 10 words in each topic)
    """
    if topk > len(topics[0]):
        raise Exception('Words in topics are less than '+str(topk))
    else:
        unique_words = set()
        for topic in topics:
            unique_words = unique_words.union(set(topic[:topk]))
        puw = len(unique_words) / (topk * len(topics))
        return puw


In [107]:
from itertools import combinations
def pairwise_jaccard_diversity(topics, topk=10):
    '''
    compute the average pairwise jaccard distance between the topics 
  
    Parameters
    ----------
    topics: a list of lists of words
    topk: top k words on which the topic diversity
          will be computed (There are 10 words in each topic)
    
    Returns
    -------
    pjd: average pairwise jaccard distance
    '''
    dist = 0
    count = 0
    for list1, list2 in combinations(topics, 2):
        js = 1 - len(set(list1).intersection(set(list2)))/len(set(list1).union(set(list2)))
        dist = dist + js
        count = count + 1
    return dist/count

In [108]:
# The first inputs the topics, the second inputs the number of words in each topic
proportion_unique_words(topics, 10)

0.68125

In [109]:
pairwise_jaccard_diversity(topics, 10)

0.9647808143217262